<a href="https://colab.research.google.com/github/vasid99/cs6910-dl/blob/main/Assignment02/Assignment02_part_C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Video of model in action](https://youtu.be/pnCPKJ0zHeo)

# Code

## Part 1: Preparation

## Preliminary checks

Make sure GPUs are enabled in Edit > Notebook Settings

In [ ]:
import cv2
print("OpenCV version",cv2.__version__,"\n")
print("NVIDIA drivers:")
!nvidia-smi

### Mount Google Drive for accessing dataset

In [ ]:
DRIVE_PATH="/content/drive"

from google.colab import drive
drive.mount(DRIVE_PATH)

fullDrivePath = lambda s:DRIVE_PATH+"/MyDrive/"+s

### A) Darknet

In [ ]:
%cd /content
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
%cd /content/darknet
!sed -i "s/GPU=0/GPU=1/" Makefile
!sed -i "s/CUDNN=0/CUDNN=1/" Makefile
!sed -i "s/CUDNN_HALF=0/CUDNN_HALF=1/" Makefile
!sed -i "s/OPENCV=0/OPENCV=1/" Makefile
!make

### B) Mask Dataset + Darknet files specific to it

First add a shortcut to [this folder](https://drive.google.com/drive/folders/1GTPvtwCJF0Cfx4uw5_TAExfaTTnGjhJU?usp=sharing) to your Google Drive (click -> Add Shortcut To Drive).<br><br>
Once done, run cells below, putting path to the shortcut folder into the `DS_PATH` variable below

In [5]:
DS_FULL_PATH=fullDrivePath("CS6910/A2/yolo-mask/yolo-mask-dataset")

!ln -s $DS_FULL_PATH/images /content/darknet/data/images

!cp $DS_FULL_PATH/train.txt  /content/darknet/data/train.txt
!cp $DS_FULL_PATH/valid.txt  /content/darknet/data/valid.txt
!cp $DS_FULL_PATH/test.txt   /content/darknet/data/test.txt 

!cp $DS_FULL_PATH/custom.data  /content/darknet/data/custom.data 
!cp $DS_FULL_PATH/custom.names /content/darknet/data/custom.names

!cp $DS_FULL_PATH/yolov4-custom-mask.cfg /content/darknet/cfg/yolov4-custom-mask.cfg
!cp $DS_FULL_PATH/yolov4-tiny-custom-mask.cfg /content/darknet/cfg/yolov4-tiny-custom-mask.cfg

### C) Pretrained weights for fresh training of mask dataset

In [ ]:
%cd /content/darknet
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

### D) Pretrained weights for finetuning/testing of mask dataset

Training for mask dataset has been done for `yolov4-tiny` for ~23000 iterations, with weights saved [here](https://drive.google.com/drive/folders/1-0Zt08WmQVeOLz7-bHJxf47CGj2aO8H_?usp=sharing).<br><br>
Download and copy them to the current runtime into `/content/darknet/backup`

### E) DOS to Unix file encoding

In [ ]:
!sudo apt install dos2unix

In [ ]:
!dos2unix /content/darknet/data/train.txt
!dos2unix /content/darknet/data/valid.txt
!dos2unix /content/darknet/data/test.txt

!dos2unix /content/darknet/data/custom.data
!dos2unix /content/darknet/data/custom.names

!dos2unix /content/darknet/cfg/yolov4-custom-mask.cfg
!dos2unix /content/darknet/cfg/yolov4-tiny-custom-mask.cfg

## Part 2: Training

### Train mask detector from scratch

In [ ]:
!/content/darknet/darknet detector train /content/darknet/data/custom.data /content/darknet/cfg/yolov4-tiny-custom-mask.cfg /content/darknet/yolov4.conv.137 -dont_show -map

### Train mask detector from pretrained mask dataset weights

Run to retrieve previous backups from Drive. If there are no backups in Drive, refer to Part 1(D)

In [ ]:
BACKUP_BASE_PATH=fullDrivePath("CS6910/A2/yolo-mask/yolo-mask-weights-backup") # ensure that the directory exists (create if not so)

In [ ]:
!cp $BACKUP_BASE_PATH/trained-weights/* /content/darknet/backup

Finetune retrieved weights

In [ ]:
!/content/darknet/darknet detector train /content/darknet/data/custom.data /content/darknet/cfg/yolov4-tiny-custom-mask.cfg /content/darknet/backup/yolov4-tiny-custom-mask_last.weights -dont_show -map

### Check mean average precision of trained model

In [ ]:
!/content/darknet/darknet detector map   /content/darknet/data/custom.data /content/darknet/cfg/yolov4-tiny-custom-mask.cfg /content/darknet/backup/yolov4-tiny-custom-mask_best.weights

### Back up weights trained so far

In [20]:
!cp -r /content/darknet/backup $BACKUP_BASE_PATH/$(date +%Y%m%d_%H%M%S)

## Part 3: Testing

### Test on sample image

Output is sent to `/content/darknet/predictions.jpg`

In [ ]:
!/content/darknet/darknet detector test /content/darknet/data/custom.data /content/darknet/cfg/yolov4-tiny-custom-mask.cfg backup/yolov4-tiny-custom-mask_last.weights -dont_show $DS_FULL_PATH/images/test/Persone.jpg

### Test on sample video

Output is sent to `-out_filename` argument (`/content/darknet/results.avi` by default)

In [ ]:
!/content/darknet/darknet detector demo /content/darknet/data/custom.data /content/darknet/cfg/yolov4-tiny-custom-mask.cfg backup/yolov4-tiny-custom-mask_last.weights -dont_show $DS_FULL_PATH/video/maskvideo1.mp4 -i 0 -out_filename results.avi